In [4]:
# import Libraries

import openai
import langchain
from pinecone import Pinecone as PineconeClient, ServerlessSpec
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAI
import os

In [5]:
from dotenv import load_dotenv
load_dotenv()

True

In [6]:
def read_doc(directory):
    file_loader=PyPDFDirectoryLoader(directory)
    documents=file_loader.load()
    return documents

In [7]:
doc=read_doc('documents/')
len(doc)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 41 0 (offset 0)
Ignoring wrong pointing object 51 0 (offset 0)


14

In [8]:
doc

[Document(page_content='Co-Brand Name \nIdeas Are Dimes A Dozen: Large Language Models For Idea \nGeneration In Innovation \nKaran Girotra, Lennart Meincke, Christian Terwiesch, and Karl T. Ulrich1\nJuly 10, 2023 \nMack Institute for Innovation Management, The Wharton School, University of Pennsylvania \nCornell Tech and Johnson College of Business, Cornell University\nAbstract \nLarge language models (LLMs) such as OpenID’s GPT series have shown remarkable capabilities in generating \nfluent and coherent text in various domains. We compare the ideation capabilities of ChatGPT-4, a chatbot based \non a state-of-the-art LLM, with those of students at an elite university. ChatGPT-4 can generate ideas much faster \nand cheaper than students, and the ideas are on average of higher quality (as measured by purchase-intent \nsurveys) and exhibit higher variance in quality. More important, the vast majority of the best ideas in the pooled \nsample are generated by ChatGPT and not by the studen

In [9]:
def chunk_data(docs,chunk_size=800,chunk_overlap=50):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=chunk_size,chunk_overlap=chunk_overlap)
    doc=text_splitter.split_documents(docs)
    return doc

In [10]:
documents=chunk_data(docs=doc)
documents

[Document(page_content='Co-Brand Name \nIdeas Are Dimes A Dozen: Large Language Models For Idea \nGeneration In Innovation \nKaran Girotra, Lennart Meincke, Christian Terwiesch, and Karl T. Ulrich1\nJuly 10, 2023 \nMack Institute for Innovation Management, The Wharton School, University of Pennsylvania \nCornell Tech and Johnson College of Business, Cornell University\nAbstract \nLarge language models (LLMs) such as OpenID’s GPT series have shown remarkable capabilities in generating \nfluent and coherent text in various domains. We compare the ideation capabilities of ChatGPT-4, a chatbot based \non a state-of-the-art LLM, with those of students at an elite university. ChatGPT-4 can generate ideas much faster \nand cheaper than students, and the ideas are on average of higher quality (as measured by purchase-intent', metadata={'source': 'documents/LLM-Ideas-Working-Paper.pdf', 'page': 0}),
 Document(page_content='surveys) and exhibit higher variance in quality. More important, the vas

In [11]:
len(documents)

48

In [12]:
## Embedding Technique Of OPENAI
embeddings=OpenAIEmbeddings(api_key=os.environ['OPENAI_API_KEY'])
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7fc6e3ee7e20>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7fc6e49c6040>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [13]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

# Check if API key is loaded (without printing the actual key for security)
if os.getenv("OPENAI_API_KEY"):
    print("OpenAI API key loaded successfully")
else:
    print("Warning: OpenAI API key not found in environment variables")

OpenAI API key loaded successfully


In [14]:
vectors=embeddings.embed_query("How are you?")
vectors

[-0.016786390884603524,
 -0.012108419068213652,
 0.006708384944812083,
 -0.02596953506456214,
 -0.016144558757618204,
 0.01760102432705511,
 -0.01113332712835445,
 -0.00991754790804511,
 -0.018131772022835137,
 -0.010423608105062689,
 0.02787034767204594,
 0.0016385245805237698,
 -0.0073008457995627956,
 -0.011682587654238595,
 0.007239131243608252,
 -0.015391639405459874,
 0.028364064119682293,
 -0.011812188640838297,
 0.014033917311814754,
 -0.020588016099571123,
 0.002419215156898743,
 0.006350439961482183,
 0.0010391208324074049,
 -0.008195708490718191,
 -0.015910042420536105,
 -0.007782220360463071,
 0.025093186600494723,
 -0.012423164188338272,
 0.022340713027705957,
 -0.025179587258227856,
 0.00562837812013177,
 0.007689648293700612,
 -0.013194597301923293,
 0.004073168609241792,
 0.00882519873096743,
 -0.022340713027705957,
 0.004057739853837834,
 -0.01047915218331048,
 0.020291784740873193,
 -0.0063566113705115085,
 0.027055714229594352,
 0.0012558936118306013,
 -0.00520563264

In [15]:

len(vectors)

1536

In [16]:
## Vector Search DB In Pinecone
# Initialize Pinecone with new SDK
pc = PineconeClient(api_key=os.environ.get('PINECONE_API_KEY'))
index_name = "langchanivector"

# Create index if it doesn't exist
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embeddings dimension
        metric='cosine',
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

In [17]:
index = PineconeVectorStore.from_documents(
    documents=doc,
    embedding=embeddings,
    index_name=index_name,
    pinecone_api_key=os.environ.get('PINECONE_API_KEY')
)


In [18]:
## Cosine Similarity Retreive Results from VectorDB
def retrieve_query(query,k=2):
    matching_results=index.similarity_search(query,k=k)
    return matching_results

In [19]:
from langchain.chains.question_answering import load_qa_chain
from langchain_openai import OpenAI

In [20]:
llm=OpenAI(model="gpt-3.5-turbo-instruct",temperature=0.5)
chain=load_qa_chain(llm,chain_type="stuff")

In [21]:
## Search answers from VectorDB
def retrieve_answers(query):
    doc_search=retrieve_query(query)
    print(doc_search)
    response=chain.run(input_documents=doc_search,question=query)
    return response

In [22]:
our_query = "tell me about the Approach"
answer = retrieve_answers(our_query)
print(answer)

[Document(page_content=' \nSecond, what is the quality distribution of the ideas generated? We are particularly interested in the extreme values – the quality of the best ideas in the three pools. We measure the quality of the ideas using the standard market research technique of eliciting consumer purchase intent in a survey. Given an estimate of the quality of each idea, we can then compare the distributional characteristics of the quality of the three pools of ideas. Third, given the performance of ChatGPT-4 in generating new product ideas, how can LLMs be used effectively in practice and what are the implications for the management of innovation? \nApproach We have over 20 years of experience teaching product design and innovation courses at Wharton, Cornell Tech, and INSEAD. We have used similar innovation challenges dozens of times with thousands of students. Most of our courses embody the innovation tournament format (Terwiesch and Ulrich 2009, 2023), in which individuals first 

/opt/anaconda3/envs/myenv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 The approach used in this study involved using over 20 years of experience teaching product design and innovation courses to generate a large set of ideas from humans. These ideas were then compared to ideas generated using LLMs, specifically ChatGPT-4. The quality distribution of the ideas was measured, with a focus on the extreme values. The study also aimed to determine how LLMs can be used effectively in practice and what implications this has for innovation management. The VIDE model was used to measure the expected future economic value of the ideas, with purchase intent expressed by a sample of consumers in the target market being the best indicator. The study obtained IRB approvals and used mTurk to evaluate all 400 ideas, with each idea being evaluated an average of 20 times by college-age individuals in the United States.
